In [1]:
# takes 30h for 6m->3m hand records, 10h per 1m, 1h per 100k.
# Read acbl_hand_records.pkl file and clean up double dummy (DD) and par errors.
# Best to re-use previous cleaned up results as performance is slow. Use previous results unless record is new.
# It's not sufficient to validate par score vul calcs by simply correcting vul because some scores would be missing.
# Creates ../acbl/acbl_hand_records_cleaned.pkl

# todo:
# acbl_hand_records.pkl are incorrect due to legacy ignoring of vul. Clean here or correct sql in previous step? 
# figure out why this line throws an exception: stdhrs.at[grp_start+handno,'Par'] = pars
# board_record_string has replaced '10' with 'T'. This is confusing as it doesn't match sql. Needed?

# next steps:
# merge-hand-records.ipynb merges acbl and tcg standardized hand records?

# previous steps:
# acbl_sql_to_hand_records.ipynb creates acbl_hand_records.pkl

# Requirements
# https://github.com/Afwas/python-dds with some of my mods.

In [2]:
import pandas as pd
import pathlib
import mlBridgeLib
from IPython.display import display # needed to define display() method in vscode

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [5]:
# takes 
acbl_hand_records_augmented_df = acblPath.joinpath('acbl_hand_records_cleaned.pkl') # tip: overwrite file with ..._dd_experiments.pkl for restart
stdhrs = pd.read_pickle(acbl_hand_records_augmented_df)
display(len(stdhrs),stdhrs)

3144755

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))"
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))"
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(140, [(3, H, , NS, 0)])","((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))"
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-460, [(5, N, , EW, 0)])","((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))"
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])","((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))"
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])","((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))"
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])","((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))"
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])","((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))"
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])","((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))"
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])","((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))"


In [6]:
# undertricks: 0 = make; 1-13 = sacrifice
# overTricks: 0-3; e.g. 1 for 4S + 1
# level: 1-7
denom = 'NSHDC' # DDS ordering
seats = ['N','E','S','W','NS','EW'] # Par score player's direction codes

# todo: change references to use mlBridgeLib
dealer_d = {'N':0, 'E':1, 'S':2, 'W':3}
vul_d = {'None':0, 'Both':1, 'N_S':2, 'E_W':3} # dds vul encoding is weird

In [7]:
# takes 1h per 100k.
# perform initialization for calculation of double dummy (all positions and all strains) and Pars.

import dds
import ctypes
import functions

starting_handno = 0 # for restarting at a particular hando (0-len(stdhrs)). normally 0.

DDdealsPBN = dds.ddTableDealsPBN()
tableRes = dds.ddTablesRes()
pres = dds.allParResults()
presmaster = dds.parResultsMaster()

mode = 0
tFilter = ctypes.c_int * dds.DDS_STRAINS
trumpFilter = tFilter(0, 0, 0, 0, 0)
line = ctypes.create_string_buffer(80)

dds.SetMaxThreads(0)
max_tables = dds.MAXNOOFTABLES
nhands = len(stdhrs)
checkpoint_after_every = 1000
#stdhrs.drop(['DDmakes_nwse','Par_nwse'],axis='columns',inplace=True)
if 'DDmakes_nwse' not in stdhrs:
    stdhrs.insert(stdhrs.columns.get_loc('DDmakes')+1,'DDmakes_nwse',None)
    #stdhrs['DDmakes_nwse'] = pd.DataFrame(tuple((0,[0]))) # stdhrs['DDmakes_nwse'].astype('object')
if 'Par_nwse' not in stdhrs:
    stdhrs.insert(stdhrs.columns.get_loc('Par')+1,'Par_nwse',None)
    #stdhrs['Par_nwse'] = pd.DataFrame(tuple((0,[0]))) # stdhrs['DDmakes_nwse'].astype('object')
dd_nwse_col_loc = stdhrs.columns.get_loc('DDmakes_nwse') # column number of 'DDmakes_nwse'
par_nwse_col_loc = stdhrs.columns.get_loc('Par_nwse') # column number of 'Par_nwse'
stdhrs

Loaded lib <CDLL 'c:\sw\bridge\dds\dds-291\vs2022\x64\Release\dds.dll', handle 7fff66360000 at 0x196acf6cf10>


,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,Par_nwse,DDmakes,DDmakes_nwse
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])",None,"((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))",None
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])",None,"((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))",None
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(140, [(3, H, , NS, 0)])",None,"((9, 7, 9, 4, 6), (4, 6, 3, 8, 6), (9, 7, 9, 5, 6), (4, 6, 3, 8, 6))",None
3,"(acbl, 2620420)",77908,100961,28,200217A_28,W,N_S,STHQJ9632DJ9CJ932SKQH8DQT86542CT76SJ76HATDAK3CAKQ84SA985432HK754D7C5,"((T, QJ9632, J9, J932), (J76, AT, AK3, AKQ84), (A985432, K754, 7, 5), (KQ, 8, QT86542, T76))","(-460, [(5, N, , EW, 0)])",None,"((1, 0, 10, 8, 1), (11, 11, 3, 5, 11), (0, 0, 10, 8, 0), (11, 11, 3, 5, 11))",None
4,"(acbl, 2620399)",77908,100961,07,200217A_07,S,Both,S2HQT762D92CAKT92SAKJ7HA8DKJ4C8643SQT854HK43DA765CQS963HJ95DQT83CJ75,"((2, QT762, 92, AKT92), (QT854, K43, A765, Q), (963, J95, QT83, J75), (AKJ7, A8, KJ4, 8643))","(-1430, [(6, S, , EW, 0)])",None,"((6, 2, 6, 1, 3), (7, 11, 6, 12, 8), (6, 2, 6, 1, 3), (7, 11, 6, 12, 8))",None
5,"(acbl, 2620396)",77908,100961,04,200217A_04,W,Both,S84HAKJ83D4CQT743SA65HQ62DQ8732CK8SJ972HT5DAKJ96C95SKQT3H974DT5CAJ62,"((84, AKJ83, 4, QT743), (J972, T5, AKJ96, 95), (KQT3, 974, T5, AJ62), (A65, Q62, Q8732, K8))","(620, [(4, H, , NS, 0)])",None,"((10, 5, 10, 6, 6), (3, 8, 3, 6, 6), (10, 5, 10, 6, 6), (3, 8, 3, 6, 6))",None
6,"(acbl, 2620410)",77908,100961,18,200217A_18,E,N_S,S872HJ8753DQ873CASAJ65HQ4DAKJ4C874S93HAK6D962CQT963SKQT4HT92DT5CKJ52,"((872, J8753, Q873, A), (93, AK6, 962, QT963), (KQT4, T92, T5, KJ52), (AJ65, Q4, AKJ4, 874))","(-120, [(1, N, , EW, 1)])",None,"((4, 4, 5, 5, 5), (9, 8, 7, 7, 8), (4, 5, 5, 5, 5), (9, 8, 7, 7, 8))",None
7,"(acbl, 2620400)",77908,100961,08,200217A_08,W,None,S953H98642DJ9CKJ5SAJT6HAJ3DT2CAT96S742HKQ7DK764CQ43SKQ8HT5DAQ853C872,"((953, 98642, J9, KJ5), (742, KQ7, K764, Q43), (KQ8, T5, AQ853, 872), (AJT6, AJ3, T2, AT96))","(-140, [(1, S, , E, 2)])",None,"((4, 6, 6, 4, 5), (8, 7, 7, 9, 8), (4, 6, 6, 4, 5), (8, 7, 7, 8, 8))",None
8,"(acbl, 2620397)",77908,100961,05,200217A_05,N,N_S,S98HAKT83D84CAQ82SQJHQJ65DT2CJ7653SAK752H72DAK96CKTST643H94DQJ753C94,"((98, AKT83, 84, AQ82), (AK752, 72, AK96, KT), (T643, 94, QJ753, 94), (QJ, QJ65, T2, J7653))","(-120, [(2, N, , EW, 0)])",None,"((5, 6, 7, 4, 5), (7, 7, 6, 8, 8), (5, 6, 7, 4, 5), (7, 7, 6, 8, 8))",None
9,"(acbl, 2620398)",77908,100961,06,200217A_06,E,E_W,SA53HJT8DQ8743CQ9SKQJT87H954DJCA62S94HAQ62DAT965C54S62HK73DK2CKJT873,"((A53, JT8, Q8743, Q9), (94, AQ62, AT965, 54), (62, K73, K2, KJT873), (KQJT87, 954, J, A62))","(-140, [(2, S, , EW, 1)])",None,"((7, 6, 5, 4, 5), (5, 7, 8, 9, 5), (7, 6, 5, 4, 5), (5, 7, 8, 9, 5))",None


In [8]:
# Perform dd and par calcs. Restartable.
# todo: implement direction in pbn. always using 'N:' for now.
# todo: some pars in acbl_hand_records.pkl are incorrect due to ignoring of vul. Need to correct pkl in previous step.
import numpy as np

for n,grp_start in enumerate(range(starting_handno,nhands,max_tables)):
    n += 1
    grp_count = min(nhands-grp_start,max_tables)
    DDdealsPBN.noOfTables = grp_count
    print(f'Processing group:{n} hands:{grp_start} to {grp_start+grp_count} of {nhands}')
    for handno in range(grp_count):
        r = stdhrs.iloc[grp_start+handno]
        nwse_tuple = tuple(r['Hands'][hand] for hand in [0,3,2,1])
        pbn = 'N:'+' '.join('.'.join([suit for suit in suits]) for suits in nwse_tuple)
        assert len(pbn) == 1+1+13*4+3*4+3, len(pbn) # 69=='N'+':'+(13 cards per hand)+(3 '.' per suit per hand)+3 spaces
        pbn_encoded = pbn.encode()
        #print(len(pbn),pbn,pbn_encoded)
        DDdealsPBN.deals[handno].cards = pbn_encoded

    # CalcAllTablesPBN will do multi-threading
    res = dds.CalcAllTablesPBN(ctypes.pointer(DDdealsPBN), mode, trumpFilter, ctypes.pointer(tableRes), ctypes.pointer(pres))

    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, line)
        print(f'CalcAllTablesPBN: DDS error: {line.value.decode("utf-8")}')
        assert False, grp_start
    
    for handno in range(0, grp_count):
        r = stdhrs.iloc[grp_start+handno]
        #display(r)
        ref = r['ref'][1]
        dealer = r['Dealer']
        assert dealer in 'NESW'
        vul = r['Vul']
        assert vul in ['None','N_S','E_W','Both'], r
        par = r['Par']
        assert type(par) is tuple, r
        assert len(par) == 2, r
        assert type(par[0]) is int, r
        assert type(par[1]) is list, r
        assert len(par[1]) > 0, r
        par_result = ctypes.pointer(presmaster)
        dd_result = ctypes.pointer(tableRes.results[handno])

        # Par calculations are not multi-threading
        res = dds.DealerParBin(dd_result, par_result, dealer_d[dealer], vul_d[vul])
        if res != dds.RETURN_NO_FAULT:
            dds.ErrorMessage(res, line)
            print(f'DealerParBin: DDS error: {line.value.decode("utf-8")}')
            assert False, r

        score = par_result.contents.score
        if score == 0:
            pars_nwse = (0, [(0, '', '', '', 0)]) # par score is for everyone to pass (1 out of 100,000)
        else:
            assert par_result.contents.number > 0, r
            pars_nwse = (score,[])
            for i in range(par_result.contents.number):
                ct = par_result.contents.contracts[i]    
                #print(f'Par[{i}]: underTricks:{ct.underTricks} overTricks:{ct.overTricks} level:{ct.level} denom:{ct.denom} seats:{ct.seats}')
                assert ct.underTricks == 0 or ct.overTricks == 0
                pars_nwse[1].append((ct.level,denom[ct.denom],'*' if ct.underTricks else '',seats[ct.seats],ct.overTricks-ct.underTricks))

        #functions.PrintPBNHand(f'Group:{n} Hand:{grp_start+handno+1}/{nhands} ACBL Ref:{ref} Vul:{vul}', DDdealsPBN.deals[handno].cards)
        #functions.PrintTable(dd_result)
        DDtable_nwse = tuple(tuple(dd_result.contents.resTable[suit][pl] for suit in [3,2,1,0,4]) for pl in range(4))
        stdhrs.iat[grp_start+handno,dd_nwse_col_loc] = DDtable_nwse # updating must use stdhrs.at instead of r.iloc or r.at.
        #print(type(np.array(pars_nwse)),np.array(pars_nwse))
        stdhrs.iat[grp_start+handno,par_nwse_col_loc] = pars_nwse # updating must use stdhrs.at instead of r.iloc or r.at.
        #display(stdhrs.head()) #.iloc[grp_start+handno])
    if n%checkpoint_after_every == 0: # save every 1000 groups
        display(stdhrs.iloc[grp_start+handno])
        acbl_hand_records_dd_experiments_df = acblPath.joinpath('acbl_hand_records_dd_experiments.pkl')
        stdhrs.to_pickle(acbl_hand_records_dd_experiments_df) # takes 40s
        print(f'Checkpoint at group {n}: df written to {acbl_hand_records_dd_experiments_df}')
        starting_handno = grp_start
starting_handno = nhands

Processing group:1 hands:0 to 40 of 3144755


C:\Users\bsali\anaconda3\envs\bridge\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


Processing group:2 hands:40 to 80 of 3144755
Processing group:3 hands:80 to 120 of 3144755
Processing group:4 hands:120 to 160 of 3144755
Processing group:5 hands:160 to 200 of 3144755
Processing group:6 hands:200 to 240 of 3144755
Processing group:7 hands:240 to 280 of 3144755
Processing group:8 hands:280 to 320 of 3144755
Processing group:9 hands:320 to 360 of 3144755
Processing group:10 hands:360 to 400 of 3144755
Processing group:11 hands:400 to 440 of 3144755
Processing group:12 hands:440 to 480 of 3144755
Processing group:13 hands:480 to 520 of 3144755
Processing group:14 hands:520 to 560 of 3144755
Processing group:15 hands:560 to 600 of 3144755
Processing group:16 hands:600 to 640 of 3144755
Processing group:17 hands:640 to 680 of 3144755
Processing group:18 hands:680 to 720 of 3144755
Processing group:19 hands:720 to 760 of 3144755
Processing group:20 hands:760 to 800 of 3144755
Processing group:21 hands:800 to 840 of 3144755
Processing group:22 hands:840 to 880 of 3144755
Pro

KeyboardInterrupt: 

In [ ]:
# takes 40s
acbl_hand_records_dd_experiments_df = acblPath.joinpath('acbl_hand_records_dd_experiments.pkl')
stdhrs.to_pickle(acbl_hand_records_dd_experiments_df)

In [ ]:
error

In [ ]:
df2 = pd.DataFrame({'A':[0]})
df2.iat[0,0] = (0,[0])

In [ ]:
df2 = pd.DataFrame({'A':[tuple((1,[1]))]})
print(type(df2.iat[0,0]),df2.iat[0,0])
df2.iat[0,0] = tuple((2,[2]))
print(type(df2.iat[0,0]),df2.iat[0,0])

In [ ]:
df2 = pd.DataFrame({'A':[0]})
x = np.zeros(1, dtype = object)
x[0] = tuple((0,[0]))
df2.iat[0,0] = *x
df2